In [1]:
import pandas as pd
from redfin import Redfin
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

client = Redfin()

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [2]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]

df = df.drop_duplicates(keep=False, subset=["FullAddress"])

df.head()

C:\Users\kayle\AppData\Local\Temp\ipykernel_18196\1303363462.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
5,38213500050160000,RES,2900,SIDNEY DR,MESQUITE,751502253,38213500050160000,2900 SIDNEY DR MESQUITE TX 75150
9,38139500580020000,RES,1507,RICHARD ST,MESQUITE,751491438,38139500580020000,1507 RICHARD ST MESQUITE TX 75149


In [3]:
addresses = [df['FullAddress']]

In [4]:
# initialize list for successful pulls
houses_list = []

for each in addresses[0][0:100]:

    # reset values
    url = ''
    address = each

    try:
        # attempt to get data from each address
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']        

        house_data = client.above_the_fold(property_id, listing_id=None)

        baths = house_data['payload']['addressSectionInfo']['baths']
        beds = house_data['payload']['addressSectionInfo']['beds']
        sq_ft_lot = house_data['payload']['addressSectionInfo']['lotSize']
        sq_ft_interior = house_data['payload']['addressSectionInfo']['sqFt']['value']
        year_built = house_data['payload']['addressSectionInfo']['yearBuilt']
        lat = house_data['payload']['addressSectionInfo']['latLong']['latitude']
        long = house_data['payload']['addressSectionInfo']['latLong']['longitude']
        price = house_data['payload']['addressSectionInfo']['priceInfo']['amount']
        price_label = house_data['payload']['addressSectionInfo']['priceInfo']['label']
        img = initial_info['payload']['preloadImageUrls']

        # add to house_data_df
        house_data = {
            'property_id': property_id,
            'full_address': address,
            'beds': beds,
            'baths': baths,
            'year_built': year_built,
            'sq_ft_interior': sq_ft_interior,
            'sq_ft_lot': sq_ft_lot,
            'price': price,
            'price_label': price_label,
            'lat': lat,
            'long': long,
            'img_url': img
        }

        # append data to successful list here
        houses_list.append(house_data)

    except Exception as e:
        # anything that doesn't have above info gets thrown as error
        print(f'error: {e} at address: {address}')

error: 'baths' at address: 9925 AUTUMN WOOD DR MESQUITE TX 75181
error: 'exactMatch' at address: 4533 CLUB ESTATES PL MESQUITE TX 75150
error: 'baths' at address: 1202 BOWERS RD SEAGOVILLE TX 75159
error: 'exactMatch' at address: 2407 E SEAGOVILLE RD SEAGOVILLE TX 75159
error: 'baths' at address: 922 AUTUMN WOOD DR MESQUITE TX 75181
error: 'baths' at address: 9924 AUTUMN WOOD DR MESQUITE TX 75181
error: 'exactMatch' at address: 2101 SILVER LEAF DR MESQUITE TX 75181
error: 'baths' at address: 9929 AUTUMN WOOD DR MESQUITE TX 75181
error: 'baths' at address: 9926 AUTUMN WOOD DR MESQUITE TX 75181
error: 'baths' at address: 1002 ARMSTRONG RD SEAGOVILLE TX 75159
error: 'exactMatch' at address: 2008 JUNIPER PASS WAY MESQUITE TX 75149
error: 'baths' at address: 9928 AUTUMN WOOD DR MESQUITE TX 75181
error: 'amount' at address: 936 KRISTOF DR SEAGOVILLE TX      
error: 'baths' at address: 9930 AUTUMN WOOD DR MESQUITE TX 75181
error: 'baths' at address: 9923 AUTUMN WOOD DR MESQUITE TX 75181
error

In [5]:
# create df
house_data_df = pd.DataFrame(
    houses_list
)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url
0,31901244,2326 LUAU ST MESQUITE TX 75150,3,1.5,1966,1356,8303,241183,Redfin Estimate,32.798774,-96.649097,[https://maps.googleapis.com/maps/api/streetvi...
1,31869232,1737 HIGHLAND ST MESQUITE TX 75149,3,1.0,1955,1454,7501,218014,Redfin Estimate,32.787427,-96.609647,[https://maps.googleapis.com/maps/api/streetvi...
2,31903524,2900 SIDNEY DR MESQUITE TX 75150,3,2.0,1972,1392,7196,241089,Redfin Estimate,32.817247,-96.644125,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...
3,31869683,1507 RICHARD ST MESQUITE TX 75149,3,1.0,1957,1313,7501,205940,Redfin Estimate,32.787413,-96.616575,[https://maps.googleapis.com/maps/api/streetvi...
4,30757826,806 BRANDON DR SEAGOVILLE TX 75159,5,2.0,2004,1504,8407,275611,Redfin Estimate,32.661589,-96.547004,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...


In [6]:
# extract zip code into a new column
house_data_df['zip_code'] = house_data_df['full_address'].str.slice(-5)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url,zip_code
0,31901244,2326 LUAU ST MESQUITE TX 75150,3,1.5,1966,1356,8303,241183,Redfin Estimate,32.798774,-96.649097,[https://maps.googleapis.com/maps/api/streetvi...,75150
1,31869232,1737 HIGHLAND ST MESQUITE TX 75149,3,1.0,1955,1454,7501,218014,Redfin Estimate,32.787427,-96.609647,[https://maps.googleapis.com/maps/api/streetvi...,75149
2,31903524,2900 SIDNEY DR MESQUITE TX 75150,3,2.0,1972,1392,7196,241089,Redfin Estimate,32.817247,-96.644125,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...,75150
3,31869683,1507 RICHARD ST MESQUITE TX 75149,3,1.0,1957,1313,7501,205940,Redfin Estimate,32.787413,-96.616575,[https://maps.googleapis.com/maps/api/streetvi...,75149
4,30757826,806 BRANDON DR SEAGOVILLE TX 75159,5,2.0,2004,1504,8407,275611,Redfin Estimate,32.661589,-96.547004,[https://ssl.cdn-redfin.com/photo/90/mbphoto/0...,75159


In [7]:
house_data_df.value_counts(['price_label'])

price_label    
Redfin Estimate    78
dtype: int64

In [8]:
house_data_df.dtypes

property_id         int64
full_address       object
beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
price_label        object
lat               float64
long              float64
img_url            object
zip_code           object
dtype: object

In [9]:
# export df as csv
house_data_df.to_csv('./redfin_house_data.csv')